#### S 0.0 Set parameter and import library

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_addons as tfa
import time
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler, MinMaxScaler,Normalizer
from tensorflow.keras.applications.vgg16 import VGG16

In [2]:
import dataset,models,utils
from external.yousefi import dataset as external_dataset

In [3]:
# define parameters 
INPUT_SHAPE = (1000,90,1) 
ACTIVATION = None

In [17]:
tf.keras.backend.clear_session()

# 1. Preparation

In [4]:
### function define
lrelu = tf.keras.layers.LeakyReLU
softmax = tf.keras.layers.Softmax
normalization = tfa.layers.InstanceNormalization
identity = tf.keras.layers.Lambda(lambda x:x)

In [5]:
# Gradient reverse layer (Yanin 2015)
@tf.custom_gradient
def grad_reverse(x):
    y = tf.identity(x)
    def custom_grad(dy):
        return -dy
    return y, custom_grad

class GradReverseLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(GradReverseLayer, self).__init__()

    def call(self, x):
        return grad_reverse(x)

In [6]:
def build_encoder():
    inputs  = tf.keras.layers.InputLayer((1000,90,1))
    # Layer 1
    conv_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides = (5,5))
    norm_1 = tfa.layers.InstanceNormalization(axis=3,center=True,scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform")
    actv_1 = tf.keras.layers.ReLU()
    pool_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,1))
    # Layer 2
    conv_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides = (3,3))
    norm_2 = tfa.layers.InstanceNormalization(axis=3,center=True,scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform")
    actv_2 = tf.keras.layers.ReLU()
    pool_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,1))
    # Layer 3
    conv_3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(2,2), strides = (2,2))
    norm_3 = tfa.layers.InstanceNormalization(axis=3,center=True,scale=True,beta_initializer="random_uniform",gamma_initializer="random_uniform")
    actv_3 = tf.keras.layers.Activation('tanh')
    pool_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,1))
    # Latent output
    latent = tf.keras.layers.Flatten()
    
    # model define
    mdl = tf.keras.models.Sequential([
        inputs,conv_1,norm_1,actv_1,pool_1,
        conv_2,norm_2,actv_2,pool_2,
        conv_3,norm_3,actv_3,pool_3,latent
    ])

    return mdl


In [7]:
def build_classifier(latent_shape,output_shape):
    
    inputs  = tf.keras.layers.InputLayer(latent_shape)
    dense_1 = tf.keras.layers.Dense(128)
    actv_1  = tf.keras.layers.LeakyReLU()
    dense_2 = tf.keras.layers.Dense(output_shape)
    actv_2  = tf.keras.layers.Softmax()
    
    mdl = tf.keras.models.Sequential([inputs,dense_1,actv_1,dense_2,actv_2])
    return mdl
    
    

In [8]:
def create_VGG(num=8,top=True):
    base_model = VGG16(include_top=False,input_shape=(1000, 90, 3))
    base_model.trainable = False
    inputs = tf.keras.layers.Input(shape=(1000, 90, 3))
    x = base_model(inputs, training=False)
    p = tf.keras.layers.GlobalAveragePooling2D()(x)
    if top == True:
        p = tf.keras.layers.Dense(128,activation='relu')(p)
        outputs = tf.keras.layers.Dense(num)(p)
        model = tf.keras.Model(inputs, outputs)
    else:
        model = tf.keras.Model(inputs, p)
    return model

In [18]:
### Model 

encoder = build_encoder()
classifier = build_classifier(encoder.output_shape[1:],output_shape=7)
model = models.define_graph([encoder,classifier],INPUT_SHAPE)

### addition for DANN (gradient reverse)
# grad_reverse_layer = GradReverseLayer()
# dann_domain_discrimination_graph = model_utilis.define_graph([encoder,grad_reverse_layer,discriminator],INPUT_SHAPE)

### VGG 
# vgg_top = create_VGG(8,False)
# _,vgg_clf,vgg_discriminator = build_model(8,normalize=False,seperate=True,latent_shape=vgg_top.output_shape[1:])

# vgg_model = model_utilis.define_graph([vgg_top,vgg_clf],(1000,90,3))

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1000, 90, 1)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 1024)              207744    
_________________________________________________________________
sequential_1 (Sequential)    (None, 7)                 132103    
Total params: 339,847
Trainable params: 339,847
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
### Sample: 5,(1000,90,1) --> 5,(7,)

model(tf.random.uniform(shape=(5,1000,90,1)))

<tf.Tensor: shape=(5, 7), dtype=float32, numpy=
array([[0.14657055, 0.14354338, 0.14357886, 0.14383782, 0.14535202,
        0.14078243, 0.13633496],
       [0.14619829, 0.14379065, 0.14338136, 0.14329432, 0.1456556 ,
        0.14105475, 0.136625  ],
       [0.14627661, 0.143331  , 0.14449358, 0.14435977, 0.14477381,
        0.14019497, 0.13657022],
       [0.14599076, 0.14417191, 0.14326803, 0.14368762, 0.14551394,
        0.14126405, 0.13610362],
       [0.14665483, 0.14306092, 0.14425969, 0.14385347, 0.14518459,
        0.14034711, 0.1366394 ]], dtype=float32)>

#### S1.3 Data

In [ ]:
# External dataset (Yousefi,2017)

folderpath = "./external/yousefi/Dataset/Data"
df = external_dataset.import_dataframe(folderpath)

In [ ]:
# DatasetObject EXT

dataset_exp1 = external_dataset.DatasetObject(df,cnn=True,sep='residual')

In [13]:
del df

In [ ]:
# Self dataset EXP1 

folderpath1 = "./data/exp_1"  # CHANGE THIS IF THE PATH CHANGED
df_exp1 = dataset.import_dataframe(folderpath1) # CHANGE THIS IF THE PATH CHANGED
# processing step, required
df_exp1 = df_exp1.drop([f"theta_{i}" for i in range(1,91)], axis=1) 
print(df_exp1.shape)

dataset_exp1 = dataset.DatasetObject(df_exp1,cnn=True,stacking=False)
ohe_y = dataset_exp1.one_hot(1)
ohe_z = dataset_exp1.one_hot(2)

In [ ]:
# Self dataset EXP2 


folderpath2 = "./data/exp_2" # CHANGE THIS IF THE PATH CHANGED
df_exp2 = dataset.import_dataframe(folderpath2)
# Preprocessing 
df_exp2['user'] = df_exp2['user'].map(lambda x: x.split('.')[0]) # processing step, required
df_exp2 = df_exp2.iloc[:,2:] # processing step, required
print(df_exp2.shape)

dataset_exp2 = dataset.DatasetObject(df_exp2,cnn=True,stacking=False)
ohe_y = dataset_exp2.one_hot(1,ohe_y)
ohe_z = dataset_exp2.one_hot(2,ohe_z)

In [ ]:
df_ = pd.concat([df_exp1,df_exp2],axis=0)
dataset_exp4 = dataset.DatasetObject(df_,cnn=True,stacking=False)
ohe_y = dataset_exp4.one_hot(1)
ohe_z = dataset_exp4.one_hot(2)

In [ ]:
del df_exp1, df_exp2

# 2. Experiment

In [21]:
tf.random.set_seed(1234)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=200)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
### single
train,test = dataset_exp1.query([6])
history = model.fit(x=train[0],y=train[1],batch_size=64,epochs=100,verbose=1,validation_data=(test[0],test[1]),callbacks=[callback])

### cross validation
# model,histories,scores = model_utilis.cross_validation(model,dataset_exp1,batch_size=64,epochs=10)

[0, 1, 2, 3, 4, 5, 7, 8, 9]
Epoch 1/100
33/33 [==============================] - 5s 146ms/step - loss: 1.9150 - accuracy: 0.1913 - val_loss: 1.7918 - val_accuracy: 0.3381
Epoch 2/100
33/33 [==============================] - 5s 142ms/step - loss: 1.5762 - accuracy: 0.4310 - val_loss: 1.3566 - val_accuracy: 0.5571
Epoch 3/100
33/33 [==============================] - 5s 143ms/step - loss: 1.1034 - accuracy: 0.6515 - val_loss: 1.0609 - val_accuracy: 0.6429
Epoch 4/100
33/33 [==============================] - 5s 142ms/step - loss: 0.7563 - accuracy: 0.7593 - val_loss: 0.7562 - val_accuracy: 0.7762
Epoch 5/100
33/33 [==============================] - 5s 142ms/step - loss: 0.4985 - accuracy: 0.8519 - val_loss: 0.8057 - val_accuracy: 0.7667
Epoch 6/100
33/33 [==============================] - 5s 142ms/step - loss: 0.3176 - accuracy: 0.9128 - val_loss: 0.6735 - val_accuracy: 0.8000
Epoch 7/100
33/33 [==============================] - 5s 144ms/step - loss: 0.1899 - accuracy: 0.9640 - val_loss: 0

33/33 [==============================] - 5s 143ms/step - loss: 2.3251e-04 - accuracy: 1.0000 - val_loss: 0.7570 - val_accuracy: 0.8333
Epoch 58/100
33/33 [==============================] - 5s 143ms/step - loss: 2.2320e-04 - accuracy: 1.0000 - val_loss: 0.7710 - val_accuracy: 0.8286
Epoch 59/100
33/33 [==============================] - 5s 143ms/step - loss: 2.1484e-04 - accuracy: 1.0000 - val_loss: 0.7543 - val_accuracy: 0.8429
Epoch 60/100
33/33 [==============================] - 5s 143ms/step - loss: 2.0613e-04 - accuracy: 1.0000 - val_loss: 0.7673 - val_accuracy: 0.8286
Epoch 61/100
33/33 [==============================] - 5s 143ms/step - loss: 1.9870e-04 - accuracy: 1.0000 - val_loss: 0.7672 - val_accuracy: 0.8333
Epoch 62/100
33/33 [==============================] - 5s 147ms/step - loss: 1.9128e-04 - accuracy: 1.0000 - val_loss: 0.7686 - val_accuracy: 0.8381
Epoch 63/100
33/33 [==============================] - 5s 145ms/step - loss: 1.8402e-04 - accuracy: 1.0000 - val_loss: 0.7695 

In [25]:
# evaluation
train,test = dataset_exp1.query([6])
cmtx = utils.evaluation(model,test[0],test[1],ohe=dataset_exp1.ohe)
cmtx

[0, 1, 2, 3, 4, 5, 7, 8, 9]
              precision    recall  f1-score   support

         bed       0.90      0.87      0.88        30
        fall       1.00      0.33      0.50        30
      pickup       0.86      1.00      0.92        30
         run       0.77      1.00      0.87        30
     sitdown       0.82      0.93      0.87        30
     standup       0.86      0.83      0.85        30
        walk       0.76      0.87      0.81        30

    accuracy                           0.83       210
   macro avg       0.85      0.83      0.82       210
weighted avg       0.85      0.83      0.82       210



,predict : bed,predict : fall,predict : pickup,predict : run,predict : sitdown,predict : standup,predict : walk
actual: bed,26,0,1,0,3,0,0
actual: fall,1,10,4,4,0,3,8
actual: pickup,0,0,30,0,0,0,0
actual: run,0,0,0,30,0,0,0
actual: sitdown,0,0,0,1,28,1,0
actual: standup,2,0,0,0,3,25,0
actual: walk,0,0,0,4,0,0,26


# 3. Save

In [ ]:
current_time = datetime.now().strftime("%Y%m%d%H%M%S")
model_name = f"model_{current_time}"
savepath = f'./saved_model/{model_name}.h5'
model.save(savepath)
print("model saved: ",savepath)

In [ ]:
cmtx.to_csv(f"./record/cmtx_{model_name}.csv")